# Hyperparameter Tuning using HyperDrive



In [ ]:
# Import Dependencies

from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn 
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import joblib
import os

## Dataset

### Overview

The dataset that we will be using for this project is the [Heart Failure Prediction](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) dataset from Kaggle. 

Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

People with cardiovascular disease or who are at high cardiovascular risk need early detection and management wherein a machine learning model can be of great help.

**12 clinical features:**

* age - Age

* anaemia - Decrease of red blood cells or hemoglobin (boolean)

* creatinine_phosphokinase - Level of the CPK enzyme in the blood (mcg/L)

* diabetes - If the patient has diabetes (boolean)

* ejection_fraction - Percentage of blood leaving the heart at each contraction (percentage)

* high_blood_pressure - If the patient has hypertension (boolean)
  
* platelets - Platelets in the blood (kiloplatelets/mL)

* serum_creatinine - Level of serum creatinine in the blood (mg/dL)

* serum_sodium - Level of serum sodium in the blood (mEq/L)
  
* sex - Woman or man (binary)
  
* smoking - If the patient smokes or not (boolean)

* time - Follow-up period (days)

In this project we will use Logistic Regression and tune its hyperparameters using HyperDrive to make prediction on the death event based on the above mentioned clinical features.


In [ ]:
ws = Workspace.from_config()

experiment_name = 'new-experiment'

experiment = Experiment(ws, experiment_name)


In [ ]:
print('Workspace name: '+ ws.name,
     'Azure region: '+ ws.location,
      'Subscription id: '+ ws.subscription_id,
     'Resource group: '+ ws.resource_group, sep="\n")

run = experiment.start_logging()

In [ ]:
# Create compute cluster

cpu_cluster_name = "new-compute"

#Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace = ws, name = cpu_cluster_name)
    print("Found existing cluster. Use it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes =4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

## Create an environment

In [ ]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

In [ ]:
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

## Hyperdrive Configuration

* `early_termination_policy` : 
 *Automatically terminate poorly performing runs with an early termination policy. Early termination improves computational efficiency.*
 <nl>
 Here Bandit Policy is used as it terminates runs where the primary metric is not within the specified slack factor/slack amount compared to the best performing run thus being highly compute saving.

* `param_sampling` : 
 *Specify the parameter sampling method to use over the hyperparameter space.*
 <nl>
 Here random sampling is used in which hyperparameter values are randomly selected from the defined search space and it It supports early termination of low-performance runs.


 

In [ ]:
# Create an early termination policy. 
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval = 2, delay_evaluation=5)


# Create the different params that we will be using during training
param_sampling = RandomParameterSampling('--C' : uniform(0.01,100),
                                        '--max_iter': choice(16,32,64,128,256))

if "training" not in os.listdir():
    os.mkdir("./training")

# Create the estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory=os.path.join('./'), 
                            compute_target=cpu_cluster_name, 
                            script='train.py', 
                            environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling = param_sampling,
                                         primary_metric_name = "Accuracy", 
                                         primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
                                         max_total_runs = 25, 
                                         max_concurrent_runs=4, 
                                         policy=early_termination_policy, 
                                         estimator=estimator)

In [ ]:
#Submit your experiment

hyperdrive_run  = experiment.submit(config = hyperdrive_run_config)

## Run Details

The `RunDetails` widget shows the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output =True)

## Best Model

The best model from the hyperdrive experiments and all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy: ', best_run_metrics['Accuracy'])

In [ ]:
# Save the best model
model = best_run.register_model(model_name='best_hyperdrive_model', model_path='outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service